
# FINN Preprocessor: Local application, Downloads MODIS raster

## 1. Setting Envoronments

### Systems settings

Most likely no need to be edited.

In [ ]:
# python libraries
import sys
import os
import re
import glob
import datetime
import subprocess
import shlex
from urllib.parse import urlparse
from importlib import reload
import gdal
import matplotlib.pylab as plt


# finn preproc codes
sys.path = sys.path + ['../code_anaconda']
import downloader
import af_import
import rst_import
import polygon_import
import run_step1
import run_step2
import export_shp
import plotter

In [ ]:
# database settings
os.environ['PGDATABASE'] = 'finn'
os.environ['PGUSER'] = 'finn'
os.environ['PGPASSWORD'] = 'finn'
os.environ['PGHOST'] = 'localhost'
os.environ['PGPORT'] = '5432'

Make sure that the PostGIS database is ready.

In [ ]:
# show info for the database
!psql postgres -c 'SELECT version();'
!pg_lsclusters

In [ ]:
# TODO i want to move this to Dockerfile somehow
# create plpython, needed only once for the database
try:
    p = subprocess.run(shlex.split("psql -d %s -c 'CREATE LANGUAGE plpython3u;'" % os.environ['PGDATABASE']), 
                       check=True, stderr=subprocess.PIPE)
except subprocess.CalledProcessError as e:
    if 'already exists' in e.stderr.decode():
        print(e.stderr.decode().replace('ERROR','OK').strip())
        pass

### Settings for ActiveFire Dataset

You specify two items related active filre.
1. `tag_af`: a string that identifies this active fire dataset through the processing.
2. `af_fnames`: list of path for the shape files you downloaded from FIRMS website  
Files you specified as `af_names` will be imported into a databse schema "af_<i>tag_af</i>", and processed in the database.  
Final output file will be "out_<i>tag_af</i>_*.csv".
3. `firstday`, `lastday`: (optional) you specify first/last year of analysis

In [ ]:
# tag to identify active fire dataset
tag_af = 'testOTS_092018'

# shp file names
af_fnames = [
    '../sample_datasets/fire/testOTS_092018/fire_archive_M6_23960.shp',
    '../sample_datasets/fire/testOTS_092018/fire_archive_V1_23961.shp',
]

# You can either set fist and last day of analysis by your self here,
# firstday = datetime.date(year, 1, 1)
# lastday = datetime.date(year+1, 1, 1)

# or use the period covered by the input active fire file (keep both variables to None)
firstday = None
lastday = None

This particular sample AF dataset are provided by the developper.  In other applications, it will be user's resoponsibility to provide shape file for active fire in specified path/name.

In [ ]:
# check input file exists
for fn in af_fnames:
    print(fn, os.path.exists(fn))
if any([not os.path.exists(_) for _ in af_fnames]):
    subprocess.run(['unzip', '-uo', '../sample_datasets/fire/testOTS_092018/DL_FIRE_V1_23961.zip',
                    '-d', '../sample_datasets/fire/testOTS_092018', 
                     ],
                  check=True)
    subprocess.run(['unzip', '-uo', '../sample_datasets/fire/testOTS_092018/DL_FIRE_M6_23960.zip',
                   '-d', '../sample_datasets/fire/testOTS_092018',],
                  check=True)


### Settings for Land Surface Datasets (land cover, vegetation continuous field, region definieons)

MODIS land cover types, MODIS vegetation continuous field will be downloaded, if needed, for the region of AF input
1. `year_rst`: MODIS raster data year to be used for the analysis

Other parameters such as `tag_lct`, `tag_vcf`, `tag_regnum` (identify landcover, vcf and region number dataset) are set automatically for MODIS dataset.  
The datasets are imported into database schema "raster", with table names "rst_<i>tag_lct</i>", or "rst_modlct_2017", for example.  
An overview raster "o_32_rst_modlct_2007" is created as well, as the real dataset is difficult to handle for QA.

In [ ]:
# MODIS raster datasets' year
year_rst = 2017
# or you can match it to years of active fire file
#year_rst = None

In [ ]:
if year_rst is None:
    if any(_ is None for _ in (firstday, lastday)):
        # read from shape file
        raise RuntimeError("year_rst speficication from AF data has not implemented yet")
    else:
        # assume first day of analysis is year of interest
        year_rst = firstday.year

In [ ]:
# tag to identify datasets, automatically set to be modlct_YYYY, modvcf_YYYY
tag_lct = 'modlct_%d' % year_rst
tag_vcf = 'modvcf_%d' % year_rst

# tag for the region number polygon
tag_regnum = 'regnum'

# definition of variables in the raster files
rasters = [
        {
            'tag': tag_lct,
            'kind': 'thematic',
            'variable': 'lct'
        },
        {
            'tag': tag_vcf,
            'kind': 'continuous',
            'variables': ['tree', 'herb', 'bare'],
        },
        {
            'tag': tag_regnum,
            'kind': 'polygons',
            'variable_in': 'region_num',
            'variable': 'regnum',
        },
]

## 2. Download raster datasets

Raster files URL and directories to save data

In [ ]:
# all downloads are stored in following dir
download_rootdir = '../downloads'

In [ ]:
# earthdata's URL for landcover and VCF
is_leap = (year_rst % 4 == 0)
url_lct = 'https://e4ftl01.cr.usgs.gov/MOTA/MCD12Q1.006/%d.01.01/' % year_rst
url_vcf = 'https://e4ftl01.cr.usgs.gov/MOLT/MOD44B.006/%d.03.%02d/' % (year_rst, 5 if is_leap else 6)

ddir_lct = download_rootdir +'/'+ ''.join(urlparse(url_lct)[1:3])
ddir_vcf = download_rootdir +'/'+ ''.join(urlparse(url_vcf)[1:3])

print('LCT downloads goes to %s' % ddir_lct)
print('VCF downloads goes to %s' % ddir_vcf)

Download land cover type raster, <b>only for the tiles needed for the active fire file</b>

In [ ]:
reload(downloader)
downloader.download_only_needed(url = url_lct, droot = download_rootdir, pnts=af_fnames[0])

Verify LCT files' checksum.  If a file is correpted, the file is downloaded again.

In [ ]:
downloader.purge_corrupted(ddir = ddir_lct, url=url_lct)

Do similar for vegetation continuous field data

In [ ]:
downloader.download_only_needed(url = url_vcf, droot = download_rootdir, pnts=af_fnames[0])

In [ ]:
downloader.purge_corrupted(ddir_vcf, url=url_vcf)

## 3. Import raster datasets

Downloaded files need preprocessing, which is to extract the only raster band needed, and also make coordinate system to be WGS84.  Intermediate files are created in following directories.

In [ ]:
workdir_lct = '../proc_rst_%s' % tag_lct
workdir_vcf = '../proc_rst_%s' % tag_vcf
workdir_regnum = '../proc_rst_%s' % tag_regnum

print('LCT preprocessing occurs in %s' % workdir_lct)
print('VCF preprocessing occurs in %s' % workdir_vcf)
print('RegNum preprocessing occurs in %s' % workdir_regnum)

### Import land cover type

First grab hdf file names from the download directory

In [ ]:
search_string = "%(ddir_lct)s/MCD12Q1.A%(year_rst)s001.h??v??.006.*.hdf" % dict(
        ddir_lct = ddir_lct, year_rst=year_rst)
fnames_lct = sorted(glob.glob(search_string))
print('found %d hdf files' % len(fnames_lct) )
if len(fnames_lct) == 0:
    raise RuntimeError("check if downloads are successful and search string to be correct: %s" % search_string)

The next command performs three tasks, "merge", "resample" and "import".  First two task creates intermediate GeoTiff files in <i>work_dir</i>.  Last task actually import the data into database's <i>raster</i> schema.

You can run only selected tasks with run_XXX flags to `False`, when you know that processing failed in the middle and you resolved the issue.

In [ ]:
reload(rst_import)

rst_import.main(tag_lct, fnames=fnames_lct, workdir = workdir_lct, run_merge=True, run_resample=True, run_import=True)

At this point you should able to see the raster in the database using QGIS.  
I am also trying to make quick check here creating simple image for QA, but use of GIS tool is encouraged.

In [ ]:
%matplotlib inline
import plotter
reload(plotter)
try:
    plotter.plot('raster.o_32_rst_%s' % tag_lct, '../code_anaconda/modlct.clr')
except Exception as e:
    print("Got this error: " + str(e))
    print("Didn't work, use QGIS!")
    pass

### Import vegetation continuous fields

Analogous steps repeated for vegetation continous fields.

In [ ]:
# grab hdf file names
search_string = "%(ddir_vcf)s/MOD44B.A%(year)s065.h??v??.006.*.hdf" % dict(
        ddir_vcf = ddir_vcf, year=year_rst)
fnames_vcf = sorted(glob.glob(search_string))
print('found %d hdf files' % len(fnames_vcf) )
if len(fnames_vcf) == 0:
    raise RuntimeError("check if downloads are successfull and search string to be correct: %s" % search_string)

In [ ]:
reload(rst_import)
rst_import.main(tag_vcf, fnames=fnames_vcf, workdir = workdir_vcf, run_merge=True, run_resample=True, run_import=True)

In [ ]:
%matplotlib inline
import plotter
reload(plotter)
try:
    plotter.plot('raster.o_32_rst_%s' % tag_vcf)
except Exception as e:
    print("Got this error: " + str(e))
    print("Didn't work, use QGIS!")
    pass

### Import countries of the world shapefile

This is actually not a raster but vector data of polygons.  But since it serves conceptually similar function as raster (specify attribute for a given geographic location), I treat it as if it is a raster dataset.  

In [ ]:
if not os.path.exists(os.path.join(workdir_regnum, 'All_Countries.shp')):
    subprocess.run(['wget', '-P', workdir_regnum, 
                    'https://s3-us-west-2.amazonaws.com/earthlab-finn/All_Countries.zip'], 
                   check=True)
    subprocess.run(['unzip', os.path.join(workdir_regnum, 'All_Countries.zip'), '-d' , workdir_regnum ], 
                  check=True)

In [ ]:
reload(polygon_import)
polygon_import.main('regnum', shpname = os.path.join(workdir_regnum, 'All_Countries.shp'))

## 4. Process active fire data

### Import active fire data

Go ahead and import into database.

<b>Be careful!!</b> The code has no safe guard and wipe the schema for the scheama "af_<i>tag_af</i>" and starts over.  

Let me think the design a bit more for now.

In [ ]:
reload(af_import)

# TODO this is destructive need to safe guard!
# tell user schema is there, list table names and # of row of each.  Ask her to delete manually or something to proceed
af_import.main(tag_af, af_fnames)

print()
for i,fn in enumerate(af_fnames):
    print(fn)
    p = subprocess.run(['psql', '-c', 'select count(*) from "af_%s".af_in_%d;' % (tag_af, i+1)], stdout=subprocess.PIPE)
    print(p.stdout.decode())


### Running "step 1" grouping points

In [ ]:
reload(run_step1)
run_step1.main(tag_af, firstday=firstday, lastday=lastday, ver='v7m', run_prep = True, run_work=True)

### Running "step 2" intersection with raster datasets

In [ ]:
reload(run_step2)

assert run_step2.ver == 'v8b'
run_step2.main(tag_af, rasters, firstday=firstday, lastday=lastday)

## 5. Export the output

Default output directory is this diretory (where you have this Jupyter Notebook file), and output file has long name of having tag of each datasets.

In [ ]:
outdir = '.'
shpname = 'out_{0}_{1}_{2}_{3}.shp'.format(tag_af, tag_lct, tag_vcf, tag_regnum)

In [ ]:
schema = 'af_' + tag_af
tblname = 'out_{0}_{1}_{2}'.format(tag_lct, tag_vcf, tag_regnum)
flds = ('v_lct', 'f_lct', 'v_tree', 'v_herb', 'v_bare', 'v_regnum')

In [ ]:
reload(export_shp)
export_shp.main(outdir, schema, tblname, flds, shpname)